In [196]:
import regex as re
import pandas as pd
import pdfplumber

In [197]:
pdf = pdfplumber.open('DataUUHabibie/uu8-1999.pdf')
page = pdf.pages[:]
text = ''

text += page[0].extract_text()

for i in page[1:]:
    text += "\n" + i.extract_text()

file = text

In [198]:
# file_path = "uu7-1999.txt"

# try:
#     with open(file_path, encoding="utf8") as file:
#         file = file.read()
# except:
#     with open(file_path) as file:
#         file = file.read()

## Remove Header

In [199]:
file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

In [200]:
file_no_header

'PRESIDEN\nREPUBLIK INDONESIA\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 8 TAHUN 1999\nTENTANG\nPERLINDUNGAN KONSUMEN\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nPRESIDEN REPUBLIK INDONESIA\nMenimbang : a. bahwa pembangunan nasional bertujuan untuk mewujudkan suatu\nmasyarakat adil dan makmur yang merata materiil dan spiritual\ndalam era demokrasi ekonomi berdasarkan Pancasila dan\nUndang-Undang Dasar 1945;\nb. bahwa pembangunan perekonomian nasional pada era globalisasi\nharus dapat mendukung tumbuhnya dunia usaha sehingga mampu\nmenghasilkan beraneka barang dan/atau jasa yang memiliki\nkandungan teknologi yang dapat meningkatkan kesejahteraan\nmasyarakat banyak dan sekaligus mendapatkan kepastian atas\nbarang dan/atau jasa yang diperoleh dari perdagangan tanpa\nmengakibatkan kerugian konsumen;\nc. bahwa semakin terbukanya pasar nasional sebagai akibat dari proses\nglobalisasi ekonomi harus tetap menjamin peningkatan\nkesejahteraan masyarakat serta kepastian atas mutu, jumlah dan\nkeamanan bara

## Pisah Penjelasan

In [201]:
penjelasan = re.search("(Agar setiap orang mengetahuinya.*|Agar setiap orang dapat mengetahuinya.*)", file_no_header, re.DOTALL).group()
badan = file_no_header.replace(penjelasan, "")

In [202]:
penjelasan = re.sub("TAMBAHAN LEMBARAN NEGARA.*", "", penjelasan, flags=re.DOTALL)

# Badan

## Ambil BAB

In [203]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB\s*[IVXLCDM]+.*?)(?=\\nBAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)(.*?)(?=\\n\p{Lu}\p{Ll})"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n.*?)(?:\\n)(\p{Lu}\p{Ll}.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)<br>
Digunakan untuk mengambil semua bab<br>
Kondisi:<br>
    a. Text dimulai dengan \n tanpa mengambil \n<br>
    b. Setelah \n dimulai dengan BAB(spasi)[Bilangan Romawi 1 digit dapat diulang beberapa kali] contoh (BAB IIV, BAB VVIX) akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBAB<br>

2. (?<=BAB)(?:\s*)([IVXLCDM]+)<br>
Digunakan untuk mengambil no bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas [Bilangan romawi 1 digit dapat diulang beberapa kali] akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>

3. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)<br>
Digunakan untuk mengambil nama bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Mengambil semua karakter berupa huruf kapital atau spasi atau \n<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \n<br>

4. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)<br>
Digunakan untuk mengambil isi bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Setelah kondisi di atas teks dimulai dengan huruf kapital atau spasi atau \n hingga berhenti pada \n tanpa diambil (nama bab)<br>
    c. Semua karakter sesudahnya diambil<br>

## Ambil Bagian

In [204]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian\s*\p{Lu}.*?)(?=\\nBagian\s*\p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_no = "(?<=Bagian)(?:\s*)(\p{Lu}\p{L}*\s*)(?=\\n)"
        pattern_nama = "(?<=Bagian)(?:\s*\p{Lu}\p{L}*\s*\\n)(.*?)(?=\\n(Pasal|Paragraf))"
        pattern_isi = "Pasal.*"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bagian]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bagian]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)<br>
Digunakan untuk mengambil semua bagian<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Bagian(spasi)[Huruf kapital] contoh (Bagian H, Bagian C, Bagian D) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBagian(spasi)[Huruf kapital]<br>

2. (?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)<br>
Digunakan untuk mengambil nama bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Bagian(spasi)[Huruf kapital][Huruf atau spasi dapat diulang beberapa kali]\n tanpa diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \nPasal<br>

3. Pasal.*<br>
Digunakan untuk mengambil isi bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal dengan diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>

## Ambil Pasal

In [205]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)<br>
Digunakan untuk mengambil semua pasal<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Pasal(spasi)[Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n contoh (Pasal 1\n, Pasal 23     \n) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. Pengambilan karakter akan berhenti ketika bertemu \nPasal atau (end of string)<br>

2. (?<=Pasal)(?:\s*)(\p{N}+)<br>
Digunakan untuk mengambil no pasal<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali] akan diambil<br>

3. (?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)<br>
Digunakan untuk mengambil isi pasal<br>
Kondisi:<br>
    a. Teks dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n akan diambil (nomor pasal)<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>

## Ambil Ayat

In [206]:
def ambil_ayat(text):
    clean_text = re.sub('(?<=ayat)(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', clean_text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)<br>
Digunakan untuk mengambil semua ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung)<br>
    c. Setelah kondisi diatas semua karakter diambil<br>
    d. Pengambilan karakter diambil hingga bertemu \n(buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) atau (end of string)<br>

2. (?<=\\n|^)(?:\()(\p{N}+)(?=\))<br>
Digunakan untuk mengambil no ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung) tanpa diambil<br>
    c. Setelah kondisi diatas dimulai oleh [nomor dapat diulang beberapa kali] diambil<br>
    d. Pengambilan karakter diambil hingga bertemu (tutup kurung)<br>

3. (?<=\(\p{N}+\)\s*)(.*)<br>
Digunakan untuk mengambil isi ayat<br>
Kondisi:<br>
    a. Text dimulai dengan (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) tanpa diambil (no ayat)<br>
    b. Setelah kondisi diatas semua karakter diambil<br>

# Penjelasan

In [207]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nUndang-undang ini dengan penempatannya dalam Lembaran Negara\nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 20 April 1999\nPRESIDEN REPUBLIK INDONESIA\nttd.\nBACHRUDDIN JUSUF HABIBIE\nDiundangkan di Jakarta\npada tanggal 20 April 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA\nttd.\nAKBAR TANJUNG\nLEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 22\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 8 TAHUN 1999\nTENTANG\nPERLINDUNGAN KONSUMEN\nI. UMUM\nPembangunan dan perkembangan perekonomian umumnya dan khususnya di\nbidang perindustrian dan perdagangan nasional telah menghasilkan berbagai variasi\nbarang dan/atau jasa yang dapat dikonsumsi. Di samping itu, globalisasi dan\nperdagangan bebas yang didukung oleh kemajuan teknologi telekomunikasi dan\ninformatika telah memperluas ruang gerak arus transaksi barang dan/atau jasa\nmelintasi batas-batas wilayah suatu n

## Ambil Pasal

In [208]:
def ambil_penjelasan_pasal(text):
    # pattern = "(?<=\\n|^|\s)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal \p{N}+\s*\\n|\sPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pattern = "(?<=\\n)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*\p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

## Ambil Ayat

In [209]:
def ambil_penjelasan_ayat(text):
    # pattern = "(?<=\\n|^|\s)(Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|\sAyat\s*\(\p{N}+\)|$)"
    pattern = "(?<=\\n|^)(\s*Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [210]:
df_badan_lst = []
lst_no_bab, lst_nama_bab, lst_isi_bab = ambil_bab(badan)

for i_bab, no_bab in enumerate(lst_no_bab):
    nama_bab = lst_nama_bab[i_bab]
    lst_no_bagian, lst_nama_bagian, lst_isi_bagian = ambil_bagian(lst_isi_bab[i_bab])

    for i_bagian, no_bagian in enumerate(lst_no_bagian):
        nama_bagian = lst_nama_bagian[i_bagian]
        lst_no_pasal, lst_isi_pasal = ambil_pasal(lst_isi_bagian[i_bagian])

        for i_pasal, no_pasal in enumerate(lst_no_pasal):
            lst_no_ayat, lst_isi_ayat = ambil_ayat(lst_isi_pasal[i_pasal])
            df_ayat = pd.DataFrame()
            df_ayat["no_ayat"] = lst_no_ayat
            df_ayat["isi_ayat"] = lst_isi_ayat
            df_ayat["no_bab"] = no_bab
            df_ayat["nama_bab"] = nama_bab
            df_ayat["no_bagian"] = no_bagian
            df_ayat["nama_bagian"] = nama_bagian
            df_ayat["no_pasal"] = no_pasal
            df_badan_lst.append(df_ayat)

df_badan = pd.concat(df_badan_lst)
df_badan = df_badan[["no_bab", "nama_bab", "no_bagian", "nama_bagian", "no_pasal", "no_ayat", "isi_ayat"]]

In [211]:
df_badan

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksud dengan:\...
0,II,ASAS DAN TUJUAN,,,2,,"Perlindungan konsumen berasaskan manfaat, kead..."
0,II,ASAS DAN TUJUAN,,,3,,Perlindungan konsumen bertujuan:\na. meningkat...
0,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,4,,"Hak konsumen adalah:\na. hak atas kenyamanan, ..."
0,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,5,,Kewajiban konsumen adalah:\na. membaca atau me...
...,...,...,...,...,...,...,...
1,XIII,SANKSI,Kedua,Sanksi Pidana,62,2,Pelaku usaha yang melanggar ketentuan sebagai...
2,XIII,SANKSI,Kedua,Sanksi Pidana,62,3,Terhadap pelanggaran yang mengakibatkan luka ...
0,XIII,SANKSI,Kedua,Sanksi Pidana,63,,Terhadap sanksi pidana sebagaimana dimaksud da...
0,XIV,KETENTUAN PERALIHAN,,,64,,Segala ketentuan peraturan perundang-undangan ...


In [255]:
df_penjelasan_lst = []

lst_no_penjelasan_pasal, lst_isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)

for i_penjelasan_pasal, no_penjelasan_pasal in enumerate(lst_no_penjelasan_pasal):
    lst_no_penjelasan_ayat, lst_isi_penjelasan_ayat = ambil_penjelasan_ayat(lst_isi_penjelasan_pasal[i_penjelasan_pasal])
    df_penjelasan_ayat = pd.DataFrame()
    df_penjelasan_ayat['penjelasan'] = lst_isi_penjelasan_ayat
    df_penjelasan_ayat['penjelasan_pasal'] = lst_isi_penjelasan_pasal[i_penjelasan_pasal]
    df_penjelasan_ayat['no_ayat'] = lst_no_penjelasan_ayat
    df_penjelasan_ayat['no_pasal'] = no_penjelasan_pasal
    df_penjelasan_lst.append(df_penjelasan_ayat)

df_penjelasan = pd.concat(df_penjelasan_lst)
df_penjelasan = df_penjelasan[['no_pasal', 'penjelasan_pasal', 'no_ayat', 'penjelasan']]

In [256]:
df_penjelasan

,no_pasal,penjelasan_pasal,no_ayat,penjelasan
0,1,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...,,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...
0,2,Perlindungan konsumen diselenggarakan sebagai ...,,Perlindungan konsumen diselenggarakan sebagai ...
0,3,Cukup jelas,,Cukup jelas
0,4,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...,,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...
0,6,Cukup jelas,,Cukup jelas
...,...,...,...,...
1,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,2,Cukup jelas
2,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,3,Cukup jelas
0,63,Cukup jelas,,Cukup jelas
0,64,Cukup jelas,,Cukup jelas


In [257]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nUndang-undang ini dengan penempatannya dalam Lembaran Negara\nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 20 April 1999\nPRESIDEN REPUBLIK INDONESIA\nttd.\nBACHRUDDIN JUSUF HABIBIE\nDiundangkan di Jakarta\npada tanggal 20 April 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA\nttd.\nAKBAR TANJUNG\nLEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 22\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 8 TAHUN 1999\nTENTANG\nPERLINDUNGAN KONSUMEN\nI. UMUM\nPembangunan dan perkembangan perekonomian umumnya dan khususnya di\nbidang perindustrian dan perdagangan nasional telah menghasilkan berbagai variasi\nbarang dan/atau jasa yang dapat dikonsumsi. Di samping itu, globalisasi dan\nperdagangan bebas yang didukung oleh kemajuan teknologi telekomunikasi dan\ninformatika telah memperluas ruang gerak arus transaksi barang dan/atau jasa\nmelintasi batas-batas wilayah suatu n

In [258]:
df_penjelasan['pasal_ayat'] = df_penjelasan['no_pasal'] + "-" + df_penjelasan['no_ayat']
df_badan['pasal_ayat'] = df_badan['no_pasal'] + "-" + df_badan['no_ayat']

In [259]:
df_left = df_badan.merge(df_penjelasan, how="left", on="pasal_ayat", suffixes=('', '_y'))
df_left = df_left.drop(['pasal_ayat'], axis=1)

In [260]:
df_left

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksud dengan:\...,1,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...,,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...
1,II,ASAS DAN TUJUAN,,,2,,"Perlindungan konsumen berasaskan manfaat, kead...",2,Perlindungan konsumen diselenggarakan sebagai ...,,Perlindungan konsumen diselenggarakan sebagai ...
2,II,ASAS DAN TUJUAN,,,3,,Perlindungan konsumen bertujuan:\na. meningkat...,3,Cukup jelas,,Cukup jelas
3,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,4,,"Hak konsumen adalah:\na. hak atas kenyamanan, ...",4,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...,,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...
4,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,5,,Kewajiban konsumen adalah:\na. membaca atau me...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
123,XIII,SANKSI,Kedua,Sanksi Pidana,62,2,Pelaku usaha yang melanggar ketentuan sebagai...,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,2,Cukup jelas
124,XIII,SANKSI,Kedua,Sanksi Pidana,62,3,Terhadap pelanggaran yang mengakibatkan luka ...,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,3,Cukup jelas
125,XIII,SANKSI,Kedua,Sanksi Pidana,63,,Terhadap sanksi pidana sebagaimana dimaksud da...,63,Cukup jelas,,Cukup jelas
126,XIV,KETENTUAN PERALIHAN,,,64,,Segala ketentuan peraturan perundang-undangan ...,64,Cukup jelas,,Cukup jelas


In [261]:
df_left[pd.isna(df_left['penjelasan'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
4,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,5,,Kewajiban konsumen adalah:\na. membaca atau me...,NaN,NaN,NaN,NaN


In [262]:
df_right = df_badan.merge(df_penjelasan, how="right", on="pasal_ayat", suffixes=('', '_y'))
df_right = df_right.drop(['pasal_ayat'], axis=1)

In [263]:
df_right

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksud dengan:\...,1,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...,,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...
1,II,ASAS DAN TUJUAN,,,2,,"Perlindungan konsumen berasaskan manfaat, kead...",2,Perlindungan konsumen diselenggarakan sebagai ...,,Perlindungan konsumen diselenggarakan sebagai ...
2,II,ASAS DAN TUJUAN,,,3,,Perlindungan konsumen bertujuan:\na. meningkat...,3,Cukup jelas,,Cukup jelas
3,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,4,,"Hak konsumen adalah:\na. hak atas kenyamanan, ...",4,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...,,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...
4,III,HAK DAN KEWAJIBAN,Kedua,Hak dan Kewajiban Pelaku Usaha,6,,Hak pelaku usaha adalah:\na. hak untuk menerim...,6,Cukup jelas,,Cukup jelas
...,...,...,...,...,...,...,...,...,...,...,...
122,XIII,SANKSI,Kedua,Sanksi Pidana,62,2,Pelaku usaha yang melanggar ketentuan sebagai...,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,2,Cukup jelas
123,XIII,SANKSI,Kedua,Sanksi Pidana,62,3,Terhadap pelanggaran yang mengakibatkan luka ...,62,Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\n...,3,Cukup jelas
124,XIII,SANKSI,Kedua,Sanksi Pidana,63,,Terhadap sanksi pidana sebagaimana dimaksud da...,63,Cukup jelas,,Cukup jelas
125,XIV,KETENTUAN PERALIHAN,,,64,,Segala ketentuan peraturan perundang-undangan ...,64,Cukup jelas,,Cukup jelas


In [264]:
df_right[pd.isna(df_right['isi_ayat'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan


In [267]:
df = df_badan.merge(df_penjelasan, how="inner", on="pasal_ayat", suffixes=('', '_y'))
df = df.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y', 'penjelasan_pasal'], axis=1)

In [268]:
df

,no_bab,nama_bab,no_bagian,nama_bagian,no_pasal,no_ayat,isi_ayat,penjelasan
0,I,KETENTUAN UMUM,,,1,,Dalam Undang-undang ini yang dimaksud dengan:\...,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...
1,II,ASAS DAN TUJUAN,,,2,,"Perlindungan konsumen berasaskan manfaat, kead...",Perlindungan konsumen diselenggarakan sebagai ...
2,II,ASAS DAN TUJUAN,,,3,,Perlindungan konsumen bertujuan:\na. meningkat...,Cukup jelas
3,III,HAK DAN KEWAJIBAN,Pertama,Hak dan Kewajiban Konsumen,4,,"Hak konsumen adalah:\na. hak atas kenyamanan, ...",Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...
4,III,HAK DAN KEWAJIBAN,Kedua,Hak dan Kewajiban Pelaku Usaha,6,,Hak pelaku usaha adalah:\na. hak untuk menerim...,Cukup jelas
...,...,...,...,...,...,...,...,...
122,XIII,SANKSI,Kedua,Sanksi Pidana,62,2,Pelaku usaha yang melanggar ketentuan sebagai...,Cukup jelas
123,XIII,SANKSI,Kedua,Sanksi Pidana,62,3,Terhadap pelanggaran yang mengakibatkan luka ...,Cukup jelas
124,XIII,SANKSI,Kedua,Sanksi Pidana,63,,Terhadap sanksi pidana sebagaimana dimaksud da...,Cukup jelas
125,XIV,KETENTUAN PERALIHAN,,,64,,Segala ketentuan peraturan perundang-undangan ...,Cukup jelas


In [269]:
def ambil_penjelasan_pasal_2(text, pasal):
    pattern = f"(?<=\\n|^|\s)(Pasal\s*{pasal}\s*\\n.*?)(?=\\nPasal {pasal}\s*\\n|\sPasal\s*{pasal}\s*\\n|$)"
    # pattern = "(?<=\\n)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*\p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [298]:
missing_penjelasan_pasal = list(df_left[pd.isna(df_left['penjelasan'])]['no_pasal'])
missing_penjelasan_ayat = list(df_left[pd.isna(df_left['penjelasan'])]['no_ayat'])

for pasal, ayat in zip(missing_penjelasan_pasal, missing_penjelasan_ayat):
    print(ambil_penjelasan_pasal_2(df_left[df_left['no_pasal_y'] == str(int(pasal) - 1)]['penjelasan_pasal'].drop_duplicates().values[0], pasal))

(['5'], ['Cukup jelas'])


In [294]:
df_left.loc[2]['penjelasan']

'Cukup jelas'

In [247]:
for i, pasal, ayat in zip(missing_penjelasan, missing_penjelasan_pasal, missing_penjelasan_ayat):
    print(i, pasal, ayat)

4 5 


## UU YANG SUDAH BISA
UU1-1999<br>
UU6-1999 (Pasalnya pake romawi) FIXED<br>
UU7-1999 (Gaada penjelasan) FIXED<br>
UU8-1999<br>
UU9-1999<br>
UU10-1999<br>
UU11-1999<br>
UU13-1999<br>
UU19-1999<br>
UU21-1999<br>

## UU YANG BELOM BISA
UU12-1999 (Penjelasan gaada ayat)<br>
UU15-1999 (Nomor ayat salah, penjelasan untuk 2 ayat kurung 2 kali)<br>
UU16-1999 (Badan gaada ayat)<br>
UU20-1999 (Penjelasan pasal ada titiknya)<br>
UU22-1999 (Penjelasan gaada ayat)<br>